In [1]:
from core import vector_store, embed_model, llm
from prompts import update_prompts

from llama_index.core import VectorStoreIndex

node_postprocessors=[
]

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5, node_postprocessors=node_postprocessors)
update_prompts(query_engine)

retriever = index.as_retriever(similarity_top_k=5, node_postprocessors=node_postprocessors)

In [12]:
# https://docs.llamaindex.ai/en/stable/examples/vector_stores/ElasticsearchIndexDemo/
query = 'сколько богатырей выходят из моря?'
# query = 'как лучше подключаться к удаленному компьютеру?'
response = retriever.retrieve(query)

response

2025-01-10 10:58:41,082 - INFO - Resurrected node <AiohttpHttpNode(http://localhost:9200)> (force=False)
2025-01-10 10:58:41,569 - INFO - POST http://localhost:9200/ascheglov_mistral/_search?_source_includes=metadata,model_text [status:200 duration:0.487s]
2025-01-10 10:58:41,572 - WARNING - Node <AiohttpHttpNode(http://localhost:9200)> has been marked alive after a successful request


[NodeWithScore(node=TextNode(id_='cd11ab06-d664-4767-9393-5d18c6d51710', embedding=None, metadata={'document_title': 'Based on the provided candidate titles and content, I would suggest a comprehensive title that captures the essence of the document in Russian:\n\n"Системы навигации и поиска на Хабре, Разработка чат-ботов, НLP и другие технологии искусственного интеллекта в команде, Обзор проектов Langchain и Llamaindex: языковые модели и внешние данные"\n\nTranslation:\n\n"Systems of navigation and search on Habr, Developing chatbots, NLP and other AI technologies in a team, Review of Langchain and Llamaindex projects: language models and external data"\n\nThis title incorporates the main themes and entities mentioned in the context, including:\n\n* Navigation and search systems on Habr\n* Development of chatbots and NLP technologies in a team\n* Projects involving Langchain and Llamaindex, focusing on language models and external data\n\nThe title is comprehensive, yet concise, and a

In [4]:
answer = query_engine.query('как лучше подключаться к удаленному компьютеру?')
# answer = query_engine.query('сколько богатырей выходят из моря?')

answer

Response(response=' Чтобы подключиться к удаленному компьютеру, используйте ssh-клавишу для этого. Полный процесс сгенерации и копирования открытого kлюча в буфер обмена описан в статье "Упрощение работы с SSH и терминалом: Криптография и безопасное хранение ключей, Установка открытого ключа SSH и его копирование в буфер обмена".\n\n   После этого можно провести подключение по команде `ssh username@ip-адрес`, где username - логин на удаленном компьютере, а ip-адрес - его IP-адрес.', source_nodes=[NodeWithScore(node=TextNode(id_='838c97fe-885b-41a9-976a-233c197ac982', embedding=None, metadata={'file_path': '/home/ubuntu/ascheglov/hackathon-ml-search/elastic/../data/texts/project-guidelines.md', 'file_name': 'project-guidelines.md', 'file_type': 'text/markdown', 'file_size': 70812, 'creation_date': '2025-01-09', 'last_modified_date': '2025-01-09', 'document_title': '"Коммит-инструкции: почему разработчиком лучше писать о результатах"\n\nBased on the provided context, I would suggest a co

In [ ]:
## We choose a model with relatively high speed and decent accuracy.
# from llama_index.core.postprocessor import SentenceTransformerRerank
# postprocessor = SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3)
# res = postprocessor.postprocess_nodes(response, query_str=query)
# res

# # RankZephyr reranker, return top 5 candidates
# from llama_index.postprocessor.rankllm_rerank import RankLLMRerank
# reranker = RankLLMRerank(model="rank_zephyr", top_n=5)
# res = reranker.postprocess_nodes(response)
# res

#
from llama_index.postprocessor.colbert_rerank import ColbertRerank
colbert_reranker = ColbertRerank(
    top_n=5,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)
res = colbert_reranker.postprocess_nodes(response, query_str=query)
res

# ##
# from llama_index.core.postprocessor import LLMRerank
# postprocessor = LLMRerank(top_n=5, llm=llm)
# res = postprocessor.postprocess_nodes(response, query_str=query)
# res


python(34204) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Batches: 100%|██████████| 1/1 [00:04<00:00,  4.20s/it]


[NodeWithScore(node=TextNode(id_='3064534c-15cf-498b-9364-fb6f3c0de73c', embedding=None, metadata={'document_title': 'Based on the provided candidate titles and content, I would suggest a comprehensive title that captures the essence of the document in Russian:\n\n"Системы навигации и поиска на Хабре, Разработка чат-ботов, НLP и другие технологии искусственного интеллекта в команде, Обзор проектов Langchain и Llamaindex: языковые модели и внешние данные"\n\nTranslation:\n\n"Systems of navigation and search on Habr, Developing chatbots, NLP and other AI technologies in a team, Review of Langchain and Llamaindex projects: language models and external data"\n\nThis title incorporates the main themes and entities mentioned in the context, including:\n\n* Navigation and search systems on Habr\n* Development of chatbots and NLP technologies in a team\n* Projects involving Langchain and Llamaindex, focusing on language models and external data\n\nThe title is comprehensive, yet concise, and a

In [16]:
import faiss

# dimensions of text-ada-embedding-002
d = 1536
faiss_index = faiss.IndexFlatL2(d)
from llama_index.vector_stores.faiss import FaissVectorStore

vector_store = FaissVectorStore(faiss_index=faiss_index)

